<a href="https://colab.research.google.com/github/MaxGhenis/random/blob/master/scf_ed_debt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Survey of Consumer Finances Education Debt

Reproduce analysis in https://www.urban.org/urban-wire/affluent-households-owe-most-student-debt, in particular [these tables](https://edit.urban.org/sites/default/files/retracted_and_corrected_numbers_tables.pdf).

## Setup

In [0]:
import pandas as pd
import numpy as np

## Utilities

In [0]:
def cond_mean(df, col, w):
    """ Conditional mean.
    """
    tmp = df[df[col] > 0]
    return (tmp[col] * tmp[w]).sum() / tmp[w].sum()

In [0]:
def weighted_quantile(values, quantiles, sample_weight=None, 
                      values_sorted=False, old_style=False):
    """ Very close to numpy.percentile, but supports weights.
    
    From https://stackoverflow.com/a/29677616/1840471.
    
    Args:
        values: numpy array with data.
        quantiles: array-like with many quantiles needed ([0, 1]).
        sample_weight: array-like of the same length as `array`.
        values_sorted: bool, if True, then will avoid sorting of
            initial array
        old_style: if True, will correct output to be consistent
            with numpy.percentile.
    
    Returns:
        numpy.array with computed quantiles.
    """
    values = np.array(values)
    quantiles = np.array(quantiles)
    if sample_weight is None:
        sample_weight = np.ones(len(values))
    sample_weight = np.array(sample_weight)
    assert np.all(quantiles >= 0) and np.all(quantiles <= 1), \
        'quantiles should be in [0, 1]'

    if not values_sorted:
        sorter = np.argsort(values)
        values = values[sorter]
        sample_weight = sample_weight[sorter]

    weighted_quantiles = np.cumsum(sample_weight) - 0.5 * sample_weight
    if old_style:
        # To be convenient with numpy.percentile
        weighted_quantiles -= weighted_quantiles[0]
        weighted_quantiles /= weighted_quantiles[-1]
    else:
        weighted_quantiles /= np.sum(sample_weight)
    return np.interp(quantiles, weighted_quantiles, values)

In [0]:
def cond_median(df, col, w):
    """ Conditional median.
    """
    tmp = df[df[col] > 0]
    return weighted_quantile(tmp[col], 0.5, tmp[w])

In [0]:
def weighted_percentrank(df, col, w):
    """Weighted percentrank.
    
    Args:
        df: A pandas DataFrame.
        col: A string indicating the column in the DataFrame to calculate.,
        w: Weight.
    Returns:
        Percent rank.
    """
    df.sort_values(by=col, inplace=True)
    return df[w].cumsum() / df[w].sum()

In [0]:
def quartile(percentrank):
    """ Get quartile from a percent rank.
    """
    return pd.cut(percentrank, [0, 0.25, 0.5, 0.75, np.inf],
                  labels=[1, 2, 3, 4])

## Load data

Linked from https://www.federalreserve.gov/econres/scfindex.htm.

`pd.read_stata` doesn't directly read zip files, so download and unzip locally.

In [7]:
!wget https://www.federalreserve.gov/econres/files/scf2016s.zip

--2019-04-28 03:57:53--  https://www.federalreserve.gov/econres/files/scf2016s.zip
Resolving www.federalreserve.gov (www.federalreserve.gov)... 132.200.148.151
Connecting to www.federalreserve.gov (www.federalreserve.gov)|132.200.148.151|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10513398 (10M) [application/x-zip-compressed]
Saving to: ‘scf2016s.zip’

scf2016s.zip        100%[===================>]  10.03M   846KB/s    in 12s     

2019-04-28 03:58:06 (846 KB/s) - ‘scf2016s.zip’ saved [10513398/10513398]



In [8]:
!unzip -o scf2016s.zip  # Force overwrite.
!rm scf2016s.zip

Archive:  scf2016s.zip
  inflating: p16i6.dta               


In [9]:
!ls

p16i6.dta  sample_data


In [0]:
scf = pd.read_stata('p16i6.dta')

## Preprocess

These columns are `EDU_LN_{1-6}: HOW MUCH STILL OWED`.

In [0]:
OUTSTANDING_STUDENT_LOAN_COLS = ['X7824', 'X7847', 'X7870', 'X7924', 'X7947', 
                                 'X7970']

In [0]:
scf['edu_loans_owed'] = scf[OUTSTANDING_STUDENT_LOAN_COLS].sum(axis=1)

Undo 5x weights assigned for multiple imputation.

In [0]:
REPLICATES = 5
scf['wt'] = scf.X42001 / REPLICATES

Add person-level weight.

In [0]:
scf['person_wt'] = scf.X101 * scf.wt

## Analyze

### Initial checks and comparison to SCF summary

In [15]:
'Total families: ' + format(scf.wt.sum() / 1e6, '.0f') + 'M'

'Total families: 126M'

In [16]:
'Total population: ' + format(scf.person_wt.sum() / 1e6, '.0f') + 'M'

'Total population: 320M'

In [17]:
('Total outstanding education loans: $' +
 format((scf.edu_loans_owed * scf.wt).sum() / 1e9, '.0f') + 'B')

'Total outstanding education loans: $961B'

[SCF summary](https://www.federalreserve.gov/publications/files/scf17.pdf) reports 22.4% (Table 4).


In [18]:
('Share of families with outstanding education loans: ' +
 format(((scf.edu_loans_owed > 0) * scf.wt).sum() / scf.wt.sum(), '.1%'))

'Share of families with outstanding education loans: 22.3%'

[SCF summary](https://www.federalreserve.gov/publications/files/scf17.pdf) reports $34.2k (Table 4).


In [19]:
('Conditional mean of outstanding education loans: $' +
 format(cond_mean(scf, 'edu_loans_owed', 'wt'), ',.0f'))

'Conditional mean of outstanding education loans: $34,245'

[SCF summary](https://www.federalreserve.gov/publications/files/scf17.pdf) reports $19.0k (Table 4).


In [20]:
('Conditional median of outstanding education loans: $' +
 format(cond_median(scf, 'edu_loans_owed', 'wt'), ',.0f'))

'Conditional median of outstanding education loans: $19,000'

### Brookings analysis

Distribution of student loan debt by income for families aged 25 or older. "For households with more than one adult, age represents the average of their ages."

Relevant fields, per https://www.federalreserve.gov/apps/scfcb:

* `X14`: Respondent age
* `X19`: Spouse age
* `X5729`: Total income
* `X101`: Total people in the household (might not be used)

Try 2x2 percentile approaches:
* Income percentiles defined from full population vs. from families age 25 or older
* Percentiles weighted by families vs. by population

In [0]:
scf['income_percentrank'] = weighted_percentrank(scf, 'X5729', 'wt')
scf['income_quartile'] = quartile(scf.income_percentrank)

scf['income_percentrank_person'] = weighted_percentrank(scf, 'X5729', 'person_wt')
scf['income_quartile_person'] = quartile(scf.income_percentrank_person)

In [0]:
scf['avg_adult_age'] = np.where(scf.X19 > 0, (scf.X14 + scf.X19) / 2, scf.X14)
scf25 = scf[scf.avg_adult_age >= 25].copy(deep=True)

In [0]:
scf25['income_percentrank_25'] = weighted_percentrank(scf25, 'X5729', 'wt')
scf25['income_quartile_25'] = quartile(scf25.income_percentrank_25)

scf25['income_percentrank_person_25'] = weighted_percentrank(scf25, 'X5729',
                                                             'person_wt')
scf25['income_quartile_person_25'] = quartile(scf25.income_percentrank_person_25)

In [0]:
def income_quartile_threshold(quartile_col):
    return (scf25.groupby(quartile_col).X5729.max())

In [25]:
quartile_methods_thresholds = pd.DataFrame({
    'income_quartile': income_quartile_threshold('income_quartile'),
    'income_quartile_person': income_quartile_threshold('income_quartile_person'),
    'income_quartile_25': income_quartile_threshold('income_quartile_25'),
    'income_quartile_person_25': income_quartile_threshold(
        'income_quartile_person_25')
}).drop([4])  # Top maximum isn't useful.

quartile_methods_thresholds.style.format('${:,}')

,income_quartile,income_quartile_person,income_quartile_25,income_quartile_person_25
1,"$26,000","$30,000","$26,000","$31,000"
2,"$50,000","$57,000","$52,000","$60,000"
3,"$95,000","$103,000","$97,000","$107,000"


In [0]:
def edu_loan_share_by_income(quartile_col):
    return (scf25.groupby(quartile_col).edu_loans_owed.sum() / 
            scf25.edu_loans_owed.sum())

In [27]:
quartile_methods = pd.DataFrame({
    'income_quartile': edu_loan_share_by_income('income_quartile'),
    'income_quartile_person': edu_loan_share_by_income('income_quartile_person'),
    'income_quartile_25': edu_loan_share_by_income('income_quartile_25'),
    'income_quartile_person_25': edu_loan_share_by_income(
        'income_quartile_person_25')
})

quartile_methods.style.format('{:.0%}')

,income_quartile,income_quartile_person,income_quartile_25,income_quartile_person_25
1,11%,13%,11%,15%
2,23%,25%,24%,26%
3,29%,27%,27%,26%
4,38%,35%,38%,33%
